In [8]:
%%capture
%load_ext dotenv
%dotenv
%pip install azure-identity azure-mgmt-compute azure-mgmt-resource


In [18]:
import os
from azure.identity import DefaultAzureCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute.models import UsageName

credential = DefaultAzureCredential()
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
compute_client = ComputeManagementClient(credential, subscription_id)
resource_client = ResourceManagementClient(credential, subscription_id)

provider = resource_client.providers.get("Microsoft.Compute")
regions = [
    resource_type.locations
    for resource_type in provider.resource_types
    if resource_type.resource_type == "virtualMachines"
][0]
# availability_sets = compute_client.availability_sets.list_by_resource_group("machine-learning")

for region in regions:
    vm_sizes = compute_client.virtual_machine_sizes.list(location=region)
    for size in vm_sizes:
        if size.name == "NC24ads_A100_v4":
            print(f"{size.name} is available in {region}")

            # # Check quotas
            # usages = compute_client.usage.list(location=region)
            # for usage in usages:
            #     if usage.name.value == "Standard_D4_v5":
            #         print(f"Quota for {size.name} in {region}: {usage.current_value}/{usage.limit}")
